In [1]:
import h5py
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.utils.data.dataloader import DataLoader
import numpy as np
import cv2 as cv

import model
import synth_text_dataset
from transforms import img_transform, char_transform, labels_transform

%matplotlib inline

In [2]:
filename = 'Project/SynthText_train.h5'
num_of_images = 998
train_dataset = synth_text_dataset.SynthTextCharactersDatasetRAM(filename, full_image_transform=img_transform, on_get_item_transform=char_transform, target_transform=labels_transform, end_idx=int(0.8*num_of_images))
test_dataset = synth_text_dataset.SynthTextCharactersDatasetRAM(filename, full_image_transform=img_transform, on_get_item_transform=char_transform, target_transform=labels_transform, start_idx=int(0.8*num_of_images))

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, shuffle=True)

In [10]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        X = X.to(device)
        pred = model(X)
        loss = loss_fn(pred, y.long())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn, device):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y.long()).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
# device = 

init_shape = (200, 100)
classifier = model.FontClassifierModel(init_shape, 1)
lr = 1e-2
epochs = 10

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(classifier.parameters(), lr=lr)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, classifier, loss_fn, optimizer)
    test_loop(test_dataloader, classifier, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.598024  [    0/24344]
loss: 1.609038  [ 6400/24344]
loss: 1.594774  [12800/24344]
loss: 1.607514  [19200/24344]
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.598493 

Epoch 2
-------------------------------
loss: 1.619916  [    0/24344]
loss: 1.608603  [ 6400/24344]
loss: 1.607057  [12800/24344]
loss: 1.612913  [19200/24344]
Test Error: 
 Accuracy: 26.6%, Avg loss: 1.600015 

Epoch 3
-------------------------------
loss: 1.619664  [    0/24344]
loss: 1.598115  [ 6400/24344]
loss: 1.613551  [12800/24344]
loss: 1.615260  [19200/24344]
Test Error: 
 Accuracy: 26.7%, Avg loss: 1.600653 

Epoch 4
-------------------------------
loss: 1.606256  [    0/24344]
loss: 1.596988  [ 6400/24344]
loss: 1.615348  [12800/24344]
loss: 1.586465  [19200/24344]
Test Error: 
 Accuracy: 19.6%, Avg loss: 1.601511 

Epoch 5
-------------------------------
loss: 1.615892  [    0/24344]
loss: 1.613585  [ 6400/24344]
loss: 1.611430  [12800/24344]
loss: 1.612163  [19200

In [13]:
init_shape = (200, 100)
classifier = model.FontClassifierModel(init_shape, 1)
lr = 1e-1
epochs = 20

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(classifier.parameters(), lr=lr)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, classifier, loss_fn, optimizer)
    test_loop(test_dataloader, classifier, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.667897  [    0/24344]
loss: 1.603152  [ 6400/24344]
loss: 1.619558  [12800/24344]
loss: 1.646321  [19200/24344]
Test Error: 
 Accuracy: 19.9%, Avg loss: 1.605470 

Epoch 2
-------------------------------
loss: 1.650752  [    0/24344]
loss: 1.611544  [ 6400/24344]
loss: 1.605981  [12800/24344]
loss: 1.610263  [19200/24344]
Test Error: 
 Accuracy: 18.7%, Avg loss: 1.629443 

Epoch 3
-------------------------------
loss: 1.586025  [    0/24344]
loss: 1.623963  [ 6400/24344]
loss: 1.639132  [12800/24344]
loss: 1.614855  [19200/24344]
Test Error: 
 Accuracy: 16.3%, Avg loss: 1.608675 

Epoch 4
-------------------------------
loss: 1.617438  [    0/24344]
loss: 1.608133  [ 6400/24344]
loss: 1.639836  [12800/24344]
loss: 1.574690  [19200/24344]
Test Error: 
 Accuracy: 19.6%, Avg loss: 1.603941 

Epoch 5
-------------------------------
loss: 1.602811  [    0/24344]
loss: 1.619037  [ 6400/24344]
loss: 1.605661  [12800/24344]
loss: 1.606055  [19200

KeyboardInterrupt: 